In [ ]:
### DAW Pipeline: Velo/Fuss + MeteoSwiss
import os
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

sns.set_style("whitegrid")

# ---------------- Configuration ----------------
PROJECT_PATH = Path("/Users/FHNW/Documents/daw/project")  # adapt if needed

try:
    os.chdir(PROJECT_PATH)
    print(f"Working directory: {os.getcwd()}")
except FileNotFoundError:
    raise FileNotFoundError(f"Path '{PROJECT_PATH}' not found!")

VELO_CSV = "converted_Velo_Fuss_Count.csv"
METEO_CSV = "ogd-smn_bas_h_historical_2020-2029.csv"
METEO_META = "ogd-smn_meta_parameters.csv"

YEARS = [2024]      # year(s) to analyze
CHUNKSIZE = 500_000 # rows per chunk when streaming Velo data

print("\nAnalyzing years:", YEARS, "with chunk size:", CHUNKSIZE, "\n")


# ---------------- MeteoSwiss: load + aggregate ----------------
def load_meteo_data(meteo_csv, meta_csv):
    """Load MeteoSwiss data and rename columns using metadata."""
    print("--> Loading MeteoSwiss data and metadata...\n")
    data = pd.read_csv(meteo_csv, sep=";", encoding="latin1")
    meta = pd.read_csv(meta_csv, sep=";", encoding="latin1")

    print("Original MeteoSwiss data (head):")
    display(data.head(3))
    print("Metadata (head):")
    display(meta.head(3))

    rename_dict = dict(
        zip(meta["parameter_shortname"], meta["parameter_description_en"])
    )
    data = data.rename(columns=rename_dict)

    data["reference_timestamp"] = pd.to_datetime(
        data["reference_timestamp"],
        format="%d.%m.%Y %H:%M",
        dayfirst=True,
        errors="coerce",
    )

    print("\nMeteoSwiss data with parsed timestamps and readable column names:")
    display(data.head(3))
    return data


def aggregate_daily_meteo(df, years):
    """Aggregate hourly MeteoSwiss data to daily temp/precip/wind."""
    print("\nAggregating MeteoSwiss data to daily level...")
    df = df[df["reference_timestamp"].dt.year.isin(years)].copy()

    daily = (
        df.groupby(df["reference_timestamp"].dt.floor("D"))
        .agg(
            temp_mean_C=("Air temperature 2 m above ground; hourly mean", "mean"),
            precip_mm=("Precipitation; hourly total", "sum"),
            wind_mean_ms=("Wind speed scalar; hourly mean in m/s", "mean"),
        )
        .reset_index()
        .rename(columns={"reference_timestamp": "date"})
    )

    print("Daily MeteoSwiss data (head):")
    display(daily.head(3))
    return daily


# ---------------- Velo/Fuss: streaming + aggregate ----------------
def aggregate_velo_streaming(velo_csv, years, chunksize):
    """
    Stream Velo/Fuss CSV and aggregate to daily totals per site.

    Returns columns: ['SiteCode', 'SiteName', 'date', 'daily_total'].
    """
    print("\n--> Loading Velo/Fuss data in streaming mode...")
    print("Aggregating daily totals while streaming...\n")

    usecols = ["Date", "SiteCode", "SiteName", "Total"]
    daily_agg = []
    chunk_count = 0

    for chunk in pd.read_csv(
        velo_csv,
        sep=";",
        encoding="utf-8",  # Basel portal is UTF-8; fixes 'Dreirosenbrücke'
        usecols=usecols,
        chunksize=chunksize,
        low_memory=False,
    ):
        chunk_count += 1

        # Parse dates and keep only valid ones in the selected years
        chunk["Date"] = pd.to_datetime(
            chunk["Date"], errors="coerce", dayfirst=True
        )
        chunk = chunk.dropna(subset=["Date"])
        chunk = chunk[chunk["Date"].dt.year.isin(years)]
        if chunk.empty:
            continue

        # Ensure numeric counts and drop negatives
        chunk["Total"] = pd.to_numeric(chunk["Total"], errors="coerce")
        chunk = chunk.dropna(subset=["Total"])
        chunk = chunk[chunk["Total"] >= 0]

        # Create daily date column and aggregate per site + date
        chunk["date"] = chunk["Date"].dt.floor("D")
        daily = (
            chunk.groupby(["SiteCode", "SiteName", "date"], as_index=False)["Total"]
            .sum()
        )
        daily_agg.append(daily)

    if not daily_agg:
        raise ValueError("No Velo/Fuss data found for requested years.")

    result = pd.concat(daily_agg, ignore_index=True)
    result = result.rename(columns={"Total": "daily_total"})

    print(f"Finished processing {chunk_count} chunks.")
    print("Daily Velo/Fuss data (head):")
    display(result.head(3))
    return result


# ---------------- Helper: missing days per year ----------------
def check_missing_days(df, col, years):
    """Count missing calendar days per year based on a datetime column."""
    out = {}
    for y in years:
        expected = pd.date_range(f"{y}-01-01", f"{y}-12-31", freq="D")
        present = df.loc[df[col].dt.year == y, col].drop_duplicates()
        missing = expected.difference(present)
        out[y] = len(missing)
    return out


# ---------------- Merge + simple feature engineering ----------------
def merge_datasets(daily_velo, daily_meteo):
    """
    Merge Velo/Fuss and Meteo data on 'date' and add weekday/weekend features.
    """
    print("\nMerging Velo/Fuss and Meteo datasets...")
    daily_velo["date"] = pd.to_datetime(daily_velo["date"])
    daily_meteo["date"] = pd.to_datetime(daily_meteo["date"])

    merged = pd.merge(daily_velo, daily_meteo, on="date", how="left")

    merged["weekday"] = merged["date"].dt.day_name()
    merged["is_weekend"] = merged["date"].dt.dayofweek >= 5

    print(f"Merged dataset shape: {merged.shape}")
    print("Merged dataset (head):")
    display(merged.head(3))
    return merged


# ---------------- Plots (time series + scatter) ----------------
def plot_weekly_bar(merged_df, years):
    """Weekly total counts across all sites."""
    print("\nGenerating weekly bar charts...")
    for year in years:
        df_year = merged_df[merged_df["date"].dt.year == year]
        weekly = (
            df_year.groupby(df_year["date"].dt.to_period("W"))["daily_total"]
            .sum()
            .reset_index()
        )
        weekly["week_start"] = weekly["date"].dt.start_time
        tick_idx = np.arange(0, len(weekly), 4)

        plt.figure(figsize=(12, 4))
        sns.barplot(data=weekly, x="week_start", y="daily_total")
        plt.title(f"Weekly total Velo/Fuss counts ({year})")
        plt.xlabel("Week start date")
        plt.ylabel("Total counts")
        plt.xticks(
            tick_idx,
            weekly["week_start"].dt.strftime("%Y-%m-%d").iloc[tick_idx],
            rotation=45,
        )
        plt.tight_layout()
        plt.show()


def plot_scatter_trends(merged_df, cols):
    """Scatterplots of daily_total vs selected meteo variables with linear trend (seaborn)."""
    print("\nGenerating scatterplots with linear trends...")
    for col in cols:
        df = merged_df[[col, "daily_total"]].dropna()

        plt.figure(figsize=(8, 4))
        sns.regplot(
            data=df,
            x=col,
            y="daily_total",
            scatter_kws={"alpha": 0.2, "s":12},
            line_kws={"color": "red", "linewidth": 1},
        )
        plt.title(f"Daily total Velo/Fuss vs {col}")
        plt.xlabel(col)
        plt.ylabel("Daily total")
        plt.tight_layout()
        plt.show()


# ---------------- Some extra analysis ----------------
def analyze_relationships(merged_df):
    """
    Print and visualize simple group statistics to answer high-level research questions.

    - Temperature bins vs mean counts
    - Precipitation categories vs mean counts
    - Weekday vs weekend differences
    """
    df = merged_df.copy()

    # --- Temperature bins ---
    temp_bins = [-20, 0, 10, 15, 20, 25, 30, 45]
    temp_labels = ["<0", "0–10", "10–15", "15–20", "20–25", "25–30", ">30"]
    df["temp_bin"] = pd.cut(
        df["temp_mean_C"],
        bins=temp_bins,
        labels=temp_labels,
        include_lowest=True,
    )

    temp_means = df.groupby("temp_bin", observed=True)["daily_total"].mean().round(1)
    print("\nMean daily_total by temperature bin:")
    print(temp_means)

    temp_means_df = temp_means.reset_index(name="mean_daily_total")

    plt.figure(figsize=(8, 4))
    sns.barplot(data=temp_means_df, x="temp_bin", y="mean_daily_total")
    plt.ylabel("Mean daily total")
    plt.xlabel("Daily mean temperature (°C bin)")
    plt.title("Mean daily counts by temperature bin")
    plt.tight_layout()
    plt.show()

    # --- Precipitation categories ---
    precip_bins = [-0.01, 0.1, 5, 1000]
    precip_labels = ["Dry", "Light rain", "Rainy"]
    df["precip_cat"] = pd.cut(
        df["precip_mm"],
        bins=precip_bins,
        labels=precip_labels,
        include_lowest=True,
    )

    precip_means = df.groupby("precip_cat", observed=True)["daily_total"].mean().round(1)
    print("\nMean daily_total by precipitation category:")
    print(precip_means)

    precip_means_df = precip_means.reset_index(name="mean_daily_total")

    plt.figure(figsize=(8, 4))
    sns.barplot(data=precip_means_df, x="precip_cat", y="mean_daily_total")
    plt.ylabel("Mean daily total")
    plt.xlabel("Precipitation category")
    plt.title("Mean daily counts by precipitation category")
    plt.tight_layout()
    plt.show()

    # --- Weekday pattern ---
    weekday_means = df.groupby("weekday")["daily_total"].mean().round(1)
    print("\nMean daily_total by weekday:")
    print(weekday_means)

    order = [
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ]
    weekday_means_df = weekday_means.loc[order].reset_index(name="mean_daily_total")

    plt.figure(figsize=(8, 4))
    sns.barplot(
        data=weekday_means_df,
        x="weekday",
        y="mean_daily_total",
    )
    plt.ylabel("Mean daily total")
    plt.xlabel("Weekday")
    plt.title("Mean daily counts by weekday")
    plt.tight_layout()
    plt.show()

    # --- Weekday vs weekend ---
    weekend_means = df.groupby("is_weekend")["daily_total"].mean().round(1)
    print("\nMean daily_total: weekday vs weekend:")
    print(weekend_means)

    weekend_means_df = weekend_means.reset_index(name="mean_daily_total")
    weekend_means_df["label"] = weekend_means_df["is_weekend"].map(
        {False: "Weekday", True: "Weekend"}
    )

    plt.figure(figsize=(5, 4))
    sns.barplot(
        data=weekend_means_df,
        x="label",
        y="mean_daily_total",
    )
    plt.ylabel("Mean daily total")
    plt.xlabel("")
    plt.title("Mean daily counts: weekday vs weekend")
    plt.tight_layout()
    plt.show()


# ---------------- Run full pipeline ----------------
def run_pipeline():
    """Execute the full DAW pipeline and save merged dataset."""
    meteo_raw = load_meteo_data(METEO_CSV, METEO_META)
    daily_meteo = aggregate_daily_meteo(meteo_raw, YEARS)

    daily_velo = aggregate_velo_streaming(VELO_CSV, YEARS, CHUNKSIZE)

    print("\nChecking missing days in both datasets:")
    print("MeteoSwiss:", check_missing_days(daily_meteo, "date", YEARS))
    print("Velo/Fuss:", check_missing_days(daily_velo, "date", YEARS))

    merged = merge_datasets(daily_velo, daily_meteo)

    plot_weekly_bar(merged, YEARS)
    plot_scatter_trends(merged, ["temp_mean_C", "precip_mm", "wind_mean_ms"])
    analyze_relationships(merged)

    merged.to_csv("merged_dataset.csv", index=False)
    print("\nMerged dataset saved as 'merged_dataset.csv'.")
    return merged


# Run pipeline
merged_df = run_pipeline()